In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [2]:
# pip install pypdf

import os
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PDF 파일 경로 설정
pdf_filepath = 'data/tutorial-korean.pdf'

# 파일 존재 여부 확인 (파일이 없으면 오류 발생)
if not os.path.exists(pdf_filepath):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pdf_filepath}")

In [3]:

try:
    # 1. PDF 파일 로드
    loader = PyPDFLoader(pdf_filepath)  # PDF 파일을 로드할 객체 생성
    docs = loader.load()  # 문서를 전체 로드

    # 총 문서 개수 출력
    print(f"총 {len(docs)}개의 문서가 로드 되었습니다.",type(docs[0]))

    #  첫 번째 문서의 메타데이터 출력
    print("첫 번째 문서 메타데이터:")
    print(json.dumps(docs[0].metadata, indent=2, ensure_ascii=False))

    # 특정 인덱스(10번째) 문서의 내용 확인 (존재할 경우)
    if len(docs) > 10:
        print("\n10번째 문서 내용:", type(docs[10]))
        print(docs[10])  # 10번째 문서 출력

    #  2. 텍스트 분할 (200자 단위, 중첩 없음)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
    split_docs = loader.load_and_split(text_splitter=text_splitter)  # 분할된 문서 로드

    # 분할된 문서 개수 출력
    print(f"\n분할된 문서의 개수: {len(split_docs)} 타입: {type(split_docs)}")

    # 10번째 분할된 문서 내용 출력 (존재할 경우)
    if len(split_docs) > 10:
        print("\n10번째 분할된 문서:")
        print(split_docs[10])

    # 3. Lazy Load 방식으로 문서 로드
    print("\nLazy Load 방식으로 문서 로드:")
    for i, doc in enumerate(loader.lazy_load()):
        if i < 5:  # 너무 많은 출력 방지 (예제: 처음 5개만 출력)
            print(json.dumps(doc.metadata, indent=2, ensure_ascii=False))

except Exception as e:
    # 오류 발생 시 메시지 출력
    print(f"오류 발생: {e}")

총 39개의 문서가 로드 되었습니다. <class 'langchain_core.documents.base.Document'>
첫 번째 문서 메타데이터:
{
  "producer": "Acrobat Distiller with ezUniHFT",
  "creator": "PScript5.dll Version 5.2",
  "creationdate": "2005-04-26T15:21:34+09:00",
  "moddate": "2005-04-26T15:21:34+09:00",
  "author": "Owner",
  "title": "<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>",
  "source": "data/tutorial-korean.pdf",
  "total_pages": 39,
  "page": 0,
  "page_label": "1"
}

10번째 문서 내용: <class 'langchain_core.documents.base.Document'>
page_content='11
그림 5 와 같이 getRoom과 setRoom 메소드들은 각각 staff 멤버의 방번호(room 
number)를 설정하고 반환하는 동작을 합니다. getRoom 메소드를 호출해 봅시다. 객체 메뉴의 
getRoom 메소드를 선택하여 실행합니다. 그러면 대화상자에서 실행 결과를 볼 수 있을 것
입니다(그림 6). 그림 6과 같이 결과의 내용이 "(unknown room)"이 됩니다. 왜냐하면, Staff 
객체에 대한 방번호를 지정하지 않았기 때문입니다.
                     
그림 6 : 메소드 호출 결과
슈퍼 클래스에서 상속된 메소드들은 서브 메뉴(inherited from Person)에서 선택하여 사용
할 수 있습니다. 객체 팝업메뉴의 상단에는 두개의 서브메뉴
3)가 있을 것입니다. 하나는 
Object 클래스로부터 상속 받은 메소드들이고 다른 하나는 Person 클래스로부터 상속 받은 
메소

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA

# 1. 문서 로딩
print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader("C:/mylangchain/langchain_basic/data/tutorial-korean.pdf")
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

# 2. 문서 분할
print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")

# 3. 벡터화
print("==> 3. 벡터화 → 로컬 임베딩으로 변환")
embeddings = OllamaEmbeddings(model="qwen3:1.7b")

# 4. 벡터스토어 저장
print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)

# 5. 검색기 생성
print("===> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)

# 6. LLM 생성기 설정 (Ollama Qwen3:1.7b 사용)
print("===> 6. 생성 → 로컬 LLM으로 답변 생성")
llm = ChatOllama(
    model="qwen3:1.7b",
    base_url="http://localhost:11434",
    temperature=0.1,
    num_predict=1500
)

# 7. QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 8. 예시 질문 실행
query = "이 튜토리얼에서 핵심 개념은 무엇인가요?"
result = qa_chain.invoke({"query": query})

print("\n[질문]")
print(query)
print("\n[답변]")
print(result["result"])


==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료
==> 2. 문서 분할 → 작은 청크로 나누기
  66개 청크 생성 완료
==> 3. 벡터화 → 로컬 임베딩으로 변환
==> 4. 저장 → FAISS 벡터스토어에 저장


C:\Users\user\AppData\Local\Temp\ipykernel_2532\1540058493.py:26: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="qwen3:1.7b")
